# Knowledge Graph Embedding (KGE) Training Notebook

In [1]:
### Imports
# Local
from model.kge_trainer import KGETrainer
from utils import IBKHDataset

# External
import time
import random
import numpy as np
import pandas as pd
import scanpy as sc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch_geometric as pyg
from torch_geometric.nn import GCNConv, HeteroConv
from torch_geometric.nn.kge import TransE
import torch_geometric.transforms as T

from tqdm.notebook import tqdm, trange
from sklearn.metrics import adjusted_rand_score

import wandb

### Parameters
wandb.login()

wandb: Currently logged in as: loicduch (loicduch-mcgill-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Seeds
pyg.seed_everything(10) # random, np, torch, torch.cuda

---
# Training

In [3]:
### Data
try:
    data = torch.load('inputs/KGE/triplets_data.pt')
except IOError:
    iKBH_dataset = IBKHDataset(data_dir='data/iBKH')
    data = iKBH_dataset.build_data()
    torch.save(data, 'inputs/KGE/triplets_data.pt')

/var/folders/zx/8f4kdwmj1bn4vsw4b_62tln40000gn/T/ipykernel_14212/3003850809.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('inputs/KGE/triplets_data.p

In [4]:
### Device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu')
# device = torch.device('cpu')
print(f'Using device: {device}')

# DATA TO DEVICE
data.to(device)

Using device: mps


Data(edge_index=[2, 2778778], edge_type=[2778778], num_nodes=145609, num_edge_types=6)

In [5]:
### Parameters
config = {
    'model': dict(
        num_nodes = data.num_nodes,
        num_relations = data.num_edge_types,
        hidden_channels = 128,
        margin = 1.0, p_norm = 1),

    'dataloader': dict(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=4096,
        shuffle=True
    ),

    'training': dict(
        lr = 0.01,
        epochs = 15,
    ),

    'device': device,
}

### Model
kge_trainer = KGETrainer(
    model := TransE(**config['model']).to(device), # Model
    train_dataloader = model.loader(**config['dataloader']), # Dataloader
    val_dataloader = None,
    device = device,
    wandb_run = wandb.init(
        project ='GraphETM',
        group = 'iBKH-Embeddings',
        name = f'iBKH-TransE_{int(time.time())}',
        config=config, save_code=True) # Start Wandb
)

### Training
kge_trainer.fit(
    epochs = config['training']['epochs'],
    optimizer = optim.Adam(kge_trainer.model.parameters(), lr=config['training']['lr']) # Optimizer
)

Epoch 0: Training:   0%|          | 0/10185 [00:00<?, ?batch/s]

train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,0.03761


In [7]:
### Extract Embeddings
rho = kge_trainer.model.node_emb.weight.detach().cpu()
torch.save(rho, 'inputs/GraphETM/embeddings.pt')

In [ ]:
#DONE